In [ ]:
!conda install -y gdown

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    faiss-proc-1.0.0           |             cuda          18 KB  conda-forge
    filelock-3.9.0             |     pyhd8ed1ab_0          13 KB  conda-forge
    gdown-4.6.2                |     pyhd8ed1ab_0          18 KB  conda-forge
    openssl-1.1.1t             |       h0b41bf4_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following NEW packages will be INSTALLED:

  filelock           conda-forge/noarch::filelock-3.9.0-p

In [ ]:
!pwd
!pip install nltk

/kaggle/working


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tensorflow as tf
import time
import re
import pickle
import unicodedata
import os

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!ls ..

input  lib  working


### Loading Data

In [ ]:
news = pd.read_csv("./Book1.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
news.head()

,Headline,Short,Unnamed: 2
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",.
1,Ralph Mata was an internal affairs lieutenant ...,Criminal complaint: Cop used his role to help ...,NaN
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",NaN
3,With a breezy sweep of his pen President Vladi...,Nina dos Santos says Europe must be ready to a...,NaN
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,NaN


In [ ]:
news.shape

(287113, 3)

In [ ]:
news=news[:100000]

In [ ]:
contractions = {"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"}

In [ ]:
def clean_words(sentence,remove_stopwords = True):
    sentence = str(sentence).lower()
    sentence = unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore') # for converting é to e and other accented chars
    sentence = re.sub(r"http\S+","",sentence)
    if True:
        sentence = sentence.split()
        new_text = []
        for word in sentence:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        sentence = " ".join(new_text)
    sentence = re.sub(r"\"", "", sentence)
    sentence = re.sub(r"\'", "", sentence)
    sentence = re.sub(r' s ', "",sentence)
    sentence = re.sub(r"&39", "", sentence) # the inshorts data has this in it
    sentence = re.sub(r"&34", "", sentence) # the inshorts data has this in it
    sentence = re.sub(r"[\[\]\\0-9()\"$#%/@;:<>{}`+=~|.!?,-]", "", sentence)
    sentence = re.sub(r"&", "", sentence)
    sentence = re.sub(r"\\n", "", sentence)
    
    if remove_stopwords:
        sentence = sentence.split()
        stops = set(stopwords.words("english"))
        sentence = [w for w in sentence if not w in stops]
        sentence = " ".join(sentence)
    sentence = sentence.strip()
    
    return sentence

In [ ]:
news['Short'] = news['Short'].apply(lambda x: clean_words(x,remove_stopwords=False))
news['Headline'] = news['Headline'].apply(lambda x: clean_words(x))

In [ ]:
document = news['Headline']
summary = news['Short']

In [ ]:
document[2]

'drunk driver killed young woman headon crash checking mobile phone jailed six years craig ecclestontodd driving home night pub received text message reading replying veered across road driving round bend smashed rachel titleys car coming way craig ecclestontodd left using mobile phone crashed headon car driven rachel titley right died later injuries headon crash took place october mr ecclestontodds car barely recognisable pictured police said ecclestontodd drunk least three four pints beer getting behind wheel found guilty causing death dangerous driving portsmouth crown court yesterday miss titley yearold solicitors clerk cowes isle wight also spent evening friends pub drunk alcohol police said driving responsibly nothing could done avoid collision added lindsay pennell prosecuting said craig ecclestontodds driving resulted tragic death young woman rachel titley death could avoided mr ecclestontodd took decision pick mobile phone whilst driving either reading replying text message di

In [ ]:
summary[2]

'craig ecclestontodd  had drunk at least three pints before driving car  was using phone when he veered across road in yarmouth isle of wight  crashed headon into yearold rachel titleys car who died in hospital  police say he would have been over legal drinkdrive limit at time of crash  he was found guilty at portsmouth crown court of causing death by dangerous driving'

In [ ]:
news.head()

,Headline,Short,Unnamed: 2
0,associated press published est october updated...,bishop john folda of north dakota is taking ti...,.
1,ralph mata internal affairs lieutenant miamida...,criminal complaint cop used his role to help c...,NaN
2,drunk driver killed young woman headon crash c...,craig ecclestontodd had drunk at least three ...,NaN
3,breezy sweep pen president vladimir putin wrot...,nina dos santos says europe must be ready to a...,NaN
4,fleetwood team still record sky bet league one...,fleetwood top of league one after win at scun...,NaN


### Preprocessing

In [ ]:
news.shape

(100000, 3)

In [ ]:
# for decoder sequence
summary = summary.apply(lambda x: '<go> ' + x + ' <stop>')
summary.head()

0    <go> bishop john folda of north dakota is taki...
1    <go> criminal complaint cop used his role to h...
2    <go> craig ecclestontodd  had drunk at least t...
3    <go> nina dos santos says europe must be ready...
4    <go> fleetwood top of league one after  win at...
Name: Short, dtype: object

#### Tokenizing the texts into integer tokens

In [ ]:
# since < and > from default tokens cannot be removed
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'

In [ ]:
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)

In [ ]:
document_tokenizer.fit_on_texts(document)
summary_tokenizer.fit_on_texts(summary)

In [ ]:
inputs = document_tokenizer.texts_to_sequences(document)
targets = summary_tokenizer.texts_to_sequences(summary)

In [ ]:
summary_tokenizer.texts_to_sequences(["that is fine"])

[[32, 13, 1195]]

In [ ]:
print(document_tokenizer)

In [ ]:
summary_tokenizer.sequences_to_texts([[18, 2, 1, 91]])

['has the <unk> during']

In [ ]:
encoder_vocab_size = len(document_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(429636, 111073)

In [ ]:
document

0        associated press published est october updated...
1        ralph mata internal affairs lieutenant miamida...
2        drunk driver killed young woman headon crash c...
3        breezy sweep pen president vladimir putin wrot...
4        fleetwood team still record sky bet league one...
                               ...                        
99995    deni kirkova published est november updated es...
99996    furious harvard business school professor gone...
99997    daily mail reporter lewis dale facing lengthy ...
99998    daily mail reporter published est february upd...
99999    daily mail reporter published est february upd...
Name: Headline, Length: 100000, dtype: object

#### Obtaining insights on lengths for defining maxlen

In [ ]:
document_lengths = pd.Series([len(x) for x in document])
summary_lengths = pd.Series([len(x) for x in summary])

In [ ]:
document_lengths = pd.Series([len(x.split()) for x in document])
summary_lengths = pd.Series([len(x.split()) for x in summary])

In [ ]:
print(document[0].split())

['associated', 'press', 'published', 'est', 'october', 'updated', 'est', 'october', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'virus', 'late', 'september', 'early', 'october', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'churches', 'took', 'communion', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'says', 'risk', 'low', 'officials', 'feel', 'important', 'alert', 'people', 'possible', 'exposure', 'diocese', 'announced', 'monday', 'bishop', 'john', 'folda', 'taking', 'time', 'diagnosed', 'hepatitis', 'diocese', 'says', 'contracted', 'infection', 'contaminated', 'food', 'attending', 'confer

In [ ]:
document_lengths.describe()

count    100000.000000
mean        376.476510
std         180.953239
min           4.000000
25%         243.000000
50%         343.000000
75%         476.000000
max        1922.000000
dtype: float64

In [ ]:
summary_lengths.describe()

count    100000.000000
mean         49.072580
std          20.131993
min           5.000000
25%          37.000000
50%          46.000000
75%          57.000000
max        1196.000000
dtype: float64

In [ ]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 512
decoder_maxlen = 64

#### Padding/Truncating sequences for identical sequence lengths

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

### Creating dataset pipeline

In [ ]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

2023-02-11 19:55:23.632739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 19:55:23.784371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 19:55:23.785422: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 19:55:23.791193: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [ ]:
inputs

<tf.Tensor: shape=(100000, 512), dtype=int32, numpy=
array([[1330,  490,  125, ...,    0,    0,    0],
       [7777, 7430, 2484, ...,    0,    0,    0],
       [2766,  748,  136, ...,    0,    0,    0],
       ...,
       [ 330,  515,  778, ..., 1062,  160,   47],
       [ 330,  515,  778, ...,    0,    0,    0],
       [ 330,  515,  778, ..., 1455, 1865, 5574]], dtype=int32)>

In [ ]:
BUFFER_SIZE =30000
BATCH_SIZE = 64

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [ ]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [ ]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

### Feed Forward Network

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


#### Encoder consisting of multiple EncoderLayer(s)

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


#### Finally, the Transformer

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [ ]:
# hyper-params
num_layers = 4
d_model = 256 #old 128
dff = 512
num_heads = 8
EPOCHS = 20 #old 20

#### Adam optimizer with custom learning rate scheduling

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


#### Defining losses and other metrics 

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [ ]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

2023-02-11 19:55:31.441846: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 879894528 exceeds 10% of free system memory.
2023-02-11 19:55:33.803948: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 227477504 exceeds 10% of free system memory.


#### Masks

In [ ]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [ ]:
checkpoint_path = "."

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


#### Training steps

In [ ]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [ ]:
def evaluate(input_document):
    input_document = document_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index["<go>"]]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index["<stop>"]:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [ ]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

In [ ]:
testsen=clean_words("Aston Villa and Reading have been charged by the Football Association in relation to crowd disturbances occurring after their FA Cup quarter-final matches. It is alleged that Villa and Reading failed to prevent spectators encroaching on the pitch following the clubs' sixth-round victories against West Brom and Bradford respectively. Both clubs have until Thursday to respond to the charge. Reading fans invaded the pitch after the FA Cup quarter-final replay against Bradford at Madejski Stadium . Aston Villa supporters mobbed Fabian Delph and Co after they beat West Brom at Villa Park . The Midlands club's 2-0 win at Villa Park had to be temporarily stopped as fans spilled onto the pitch towards the end of the game and the full-time whistle prompted a mass pitch invasion, as Tim Sherwood's side booked their place at Wembley. Seats were also thrown from the stand holding West Brom's supporters and some Baggies players were confronted by celebrating fans. West Brom expressed their 'deep concern' for the safety of the players after the match and manager Tony Pulis described the scenes as 'disgraceful'. A shirtless man hangs from the crossbar after Villa beat the Baggies in the FA Cup quarter-final . Hundreds of Villa fans celebrated victory over their local rivals by storming onto the pitch . Reading fans also invaded the pitch after the Royals beat Bradford 3-0 in a replay at the Madejski Stadium and one supporter entered the pitch during the contest. Reading lost 2-1 after extra-time on Saturday to Arsenal, who will play Villa in the FA Cup final after Sherwood's side beat Liverpool 2-1. Police formed a line to separate the Reading supporters on the pitch from the Bradford fans in the stands . Hundreds of Reading fans surrounded players from both sides after the Royals won a place in the semi-finals .")

In [ ]:
testsen

'aston villa reading charged football association relation crowd disturbances occurring fa cup quarterfinal matches alleged villa reading failed prevent spectators encroaching pitch following clubs sixthround victories west brom bradford respectively clubs thursday respond charge reading fans invaded pitch fa cup quarterfinal replay bradford madejski stadium aston villa supporters mobbed fabian delph co beat west brom villa park midlands clubs win villa park temporarily stopped fans spilled onto pitch towards end game fulltime whistle prompted mass pitch invasion tim sherwoods side booked place wembley seats also thrown stand holding west broms supporters baggies players confronted celebrating fans west brom expressed deep concern safety players match manager tony pulis described scenes disgraceful shirtless man hangs crossbar villa beat baggies fa cup quarterfinal hundreds villa fans celebrated victory local rivals storming onto pitch reading fans also invaded pitch royals beat bradfo

In [ ]:
summarize(
    testsen)

2023-02-11 20:06:13.031859: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 439947264 exceeds 10% of free system memory.


'the yearold was given a chance to return home after a brief encounter with a man in the first half of his career he was sent off for a second yellow card for the second time in a row the yearold is now in charge of the possibility of a fine'

## 

In [ ]:
!pip install rouge
from rouge import Rouge

In [ ]:
res=vnews[20:120]
res['pred']=res['article'].apply(summarize)
rouge = Rouge()
rouge.get_scores(res['pred'].tolist(), res['highlights'].tolist(), avg=True)